In [1]:
import requests
import csv
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os

linkDizisi = ["https://www.bbc.com/news/world-60525350","https://www.bbc.com/news/coronavirus",
"https://www.bbc.com/news/science-environment-56837908","https://www.bbc.com/news/world/africa",
"https://www.bbc.com/news/world/asia","https://www.bbc.com/news/world/asia/china","https://www.bbc.com/news/world/asia/india",
"https://www.bbc.com/news/world/australia","https://www.bbc.com/news/world/europe","https://www.bbc.com/news/world/latin_america",
"https://www.bbc.com/news/world/middle_east","https://www.bbc.com/news/world/us_and_canada","https://www.bbc.com/news/england",
"https://www.bbc.com/news/northern_ireland","https://www.bbc.com/news/scotland","https://www.bbc.com/news/wales",
"https://www.bbc.com/news/world/europe/isle_of_man","https://www.bbc.com/news/world/europe/jersey","https://www.bbc.com/news/politics",
"https://www.bbc.com/news/politics/parliaments","https://www.bbc.com/news/politics/uk_leaves_the_eu","https://www.bbc.com/news/business-45489065",
"https://www.bbc.com/news/business-15521824","https://www.bbc.com/news/business/companies","https://www.bbc.com/news/business-22434141",
"https://www.bbc.com/news/business-11428889","https://www.bbc.com/news/business/economy","https://www.bbc.com/news/business/global_car_industry",
"https://www.bbc.com/news/business/business_of_sport","https://www.bbc.com/news/technology","https://www.bbc.com/news/science_and_environment",
"https://www.bbc.com/news/stories","https://www.bbc.com/news/entertainment_and_arts","https://www.bbc.com/news/health"
]

for tumlinkler in linkDizisi:
    # Dosyanın olup olmadığını kontrol etmek için(csv oluştururken başlığı 2 kez basmasın diye) #
    dosyaYolu= "C:\\Users\\berka\\Desktop\\bbcCombo.csv"
    dosyaVarmi = os.path.isfile(dosyaYolu)
    print(tumlinkler)


    # Ana sayfadaki haberlerin adresi #
    url=tumlinkler

    # Bağlantının düzgün sağlanması için tarayıcı kullanıcı bilgisi #
    headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36"}

    # Ana sayfaya bilgimizi kullanarak bağlanma #
    sayfa = requests.get(url, headers=headers)

    # Ana sayfa kaynak kodlarının alınması #
    icerik= BeautifulSoup(sayfa.content,'html.parser')


    # Ana sayfadaki haberlerin hypertext(href) değerlerini çekmek #
    haberlerinLinki = ""
    for linkal in icerik.find_all(
        'a', href=True, 
        class_='qa-heading-link lx-stream-post__header-link'):
        haberlerinLinki+=str(linkal['href']+" ")

    # Ana sayfadaki haberlerin hypertext(href) değerlerini boşlukla ayırarak diziye atmak #
    ayrikhaberler=haberlerinLinki.split(" ")

    # Ana sayfadaki haberlerin href değerlerini içeren diziden article olanları çıkarıp sadece haberleri alıp bir diziye atmak
    articleharic=" "
    linkDizisi=[]
    for q in ayrikhaberler:
        if q.startswith("/news"):
            a="https://www.bbc.com"+q
            linkDizisi.append("https://www.bbc.com"+q)



    # Link dizisinden tekrar edenleri çıkarmak için satır (html kısmında before: , after: şeklinde tanımladıkları için bazı linkler
    #iki kez geliyordu onun önüne geçtim)
    tekrarEtmeyenLinkDizisi = [] 
    [tekrarEtmeyenLinkDizisi.append(x) for x in linkDizisi if x not in tekrarEtmeyenLinkDizisi] 



    # -------------------------2.KISIM-----HABERİN İÇİNE GİRİP HABER TÜRÜNÜ VE İÇERİĞİNİ ALMAK-----2.KISIM-------------------------# 

    # Haber türü ve haber içeriğinin csv dosyasına atılmasından önce csv oluşturulur ve başlık ayarlanır
    if(dosyaVarmi==0):  
        with open('bbcCombo.csv', 'a',encoding='utf-8', newline='') as file:
            fieldnames = ['haberIcerigi', 'haberTuru']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()

    # Haberlerin linkleri içeren diziyi oluşturup tek tek dizi elemanlarına bağlanıp haberlerin içeriklerini ve türlerini çekmek
    for low in range(len(tekrarEtmeyenLinkDizisi)):
        url2=tekrarEtmeyenLinkDizisi[low]
        sayfa2 = requests.get(url2, headers=headers)

        #Haberin Sayfa kaynak kodlarının alınması
        icerik2= BeautifulSoup(sayfa2.content,'html.parser')



        # ------------------Haber içeriğini çekme---------------------#

        # [] ile birlikte dizi olarak haber içeriğinin alınması
        etiketliVeri= icerik2.find_all(class_='ssrcss-1q0x1qg-Paragraph eq5iqo00')
        
        # [] olmadan haberin içeriğinin alınması
        haberIcerigi = ""
        for i in etiketliVeri[:-15]:
            etiketsizVeri=i.string
            haberIcerigi+=str(etiketsizVeri)
            
        if haberIcerigi is None:
            continue

        # Haberleri gereksiz sözcüklerden arındırma işlemi

        istenmeyenKelime=[
            "This","video",
            "'s",".",",","''","’","-","_",
            "”","“","``","yet","?","!",":",
            "...",";","&","%","None ","(",
            ")","$","'n","n'","'ll","ca"
            ,"n't","'","But","but"
        ]
        gereksizSozcuk = stopwords.words('english')
        gereksizSozcuk.extend(istenmeyenKelime)
        Kelimeler = word_tokenize(haberIcerigi)

        FiltrelenmisCumle = [w for w in Kelimeler if not w.lower() in gereksizSozcuk]
        FiltrelenmisCumle = []

        for kelime in Kelimeler:
            if kelime not in gereksizSozcuk:
                FiltrelenmisCumle.append(kelime)
                FiltrelenmisCumle.append(" ")
        bosCumle=" "
        for tara in FiltrelenmisCumle:   
            bosCumle+=str(tara) 

        # Videolu haberlerin başlarında played sonlarında none kelimeleri oluyordu onları atmak için değiştirme işlemi
        bosCumle = bosCumle.replace("played", "")
        bosCumle = bosCumle.replace("By", "")
        bosCumle = bosCumle.replace("by", "")
        bosCumle = bosCumle.replace("None", "")     
        bosCumle = bosCumle.replace(".", "")     
        bosCumle = bosCumle.replace(",", " ")     
        
        # ------------------Haber türünü çekme---------------------#

        # Haberin türünün alınması
        haberinTuru = ""

        url2=tumlinkler
        arindirilmisUrl = url2.replace("https://www.bbc.com/news/","")

        if "/" in arindirilmisUrl:
            for i in arindirilmisUrl.split("/"):
                haberinTuru=i
                break
        elif "-" in arindirilmisUrl:
            for i in arindirilmisUrl.split("-"):
                haberinTuru=i
                break
        elif "_" in arindirilmisUrl:
            for i in arindirilmisUrl.split("_"):
                haberinTuru=i
                break
        else:
            if arindirilmisUrl=="coronavirus":
                haberinTuru="health"
            else:
                haberinTuru=arindirilmisUrl

        

        # Haber türü ve haber içeriğinin csv dosyasına atılması dataset oluşturmak için #
        if haberIcerigi =="" or haberinTuru =="": 
            continue
        else:
            with open('bbcCombo.csv', 'a',encoding='utf-8', newline='') as file:
                fieldnames = ['haberIcerigi', 'haberTuru']
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writerow({'haberIcerigi': bosCumle, 'haberTuru': haberinTuru})    


        print(bosCumle)
        print(haberinTuru)

https://www.bbc.com/news/world-60525350
 Two British men captured Russian forces fighting Ukraine taken care Russia said Aiden Aslin 28 Shaun Pinner 48 captured last week defending south-eastern city Mariupol In footage shown Russian state TV appeared facial bruising prompting families call Geneva Convention respectedRussian foreign ministry spokeswoman Maria Zakharova said Do worry Russian side taking care She added They fed watered given necessary assistance Just like foreigners surrendered detained Ms Zakharova also referenced described call Prime Minister Boris Johnson British detainees shown mercy said turn Britain show mercy citizens Ukraine stopping supply weapons Ukrainian government Mr Johnson urged Russia treat captured British fighters humanely compassionately The Geneva Convention states prisoners war must protected acts violence intimidation insults public curiosity Both Mr Aslin originally Newark Nottinghamshire Mr Pinner originally Bedfordshire lived Ukraine since 2018Th

 Thousands civilians remain trapped besieged city Mariupol evacuation attempt get stalledOn 56th day Russian invasion hoped 6 000 people would able flee besieged city convoy busesBut thousand able get end Mariupol officials sayAs mass evacuation plans stalled Ukrainian forces holding steelworks city ignored latest Russian ultimatum surrenderMariupol defenders holed network bunkers tunnels underneath city massive Azovstal steel plant reportedly 1 000 civilians alongside themThis But fighters say running supplies wo able hold much longerFor weeks Russian troops trying take full control city would free military resources It would also complete land bridge enabling Russian forces annexed Crimean peninsula link separatist forces eastThis For relatives Mariupol remaining defenders bitter moment fear mixed prideOlena Nikulina 26-year-old cousin one Ukrainian fighters holding I would never imagined cousin become true hero Ukraine Mariupol I dream day return told Mark Lowen Kyiv I thinking woul

 BBC News NI outlines latest data coronavirus Covid-19 vaccinations across Northern Ireland Republic IrelandNine Covid-19-related deaths reported Northern Ireland ThursdayThe total number deaths linked Covid-19 Northern Ireland since start pandemic 3 402 Deaths measured recording died within 28 days receiving positive result test coronavirus Another 752 cases coronavirus reported Thursday 588 WednesdayThe data includes cases confirmed samples taken previous days necessarily latest reporting periodSince 5 January people Northern Ireland positive lateral flow test LFT longer need polymerase chain reaction PCR test confirm resultHowever Department Health said extremely important people report LFT test results positiveOn Thursday 387 people Covid-19 hospital Northern Ireland five people ICUThat figure Wednesday 380 people Covid-19 hospital Northern Ireland five people ICULast updated 21 April 14:20 BSTSource Department Health Northern IrelandA total 3 778 302 vaccines Covid-19 administered

 Boris Johnson says MPs full facts Partygate ordering inquiry whether misled lockdown partiesMPs set vote Thursday Labour plan Commons committee investigate past comments Whitehall gatheringsBut ministers say vote wait probes Met Police civil servant Sue Gray finishedMr Johnson miss debate India official visitSpeaking trip said keen every possible form scrutiny wanted focus amazing opportunities developing UK IndiaEarlier Mr Johnson told reporters would fight next general election would say circumstances would resignAlong wife Carrie Chancellor Rishi Sunak PM fined breaking Covid laws June 2020 birthday party No 10He previously told MPs Covid laws broken Downing Street leading opposition parties accuse misled ParliamentMr Johnson denied knowingly misleading MPs charge resigning matter ministerial rulebookLabour plan backed opposition parties Commons Privileges Committee launch inquiry whether police investigation finishedSome rebel Tory MPs threatening vote Labour government tabled ame

https://www.bbc.com/news/science-environment-56837908
 Marine scientists listening secret world elusive species dolphin examining unique sounds attempt better protect themThe Atlantic white sided dolphin prefers deep ocean coastal waters rarely seen humans The species poorly understood science accurate estimates population size even though hunted Faroe IslandsIn fact climate change mass stranding hunting vulnerable becoming fishing -catch difficult experts definitive view whether population size healthy drawn scientists Scottish Association Marine Science SAMS study Atlantic white sided dolphin depth listening themLike dolphins make echolocation clicks high frequency pulse sound And time takes sound come back tells far away target use finding food navigation orientation Their whistling thought used social behaviourUnder sea Scotland various research groups moored number underwater microphones known hydrophones record noises marine creatures Traditionally make sense data person expert e

...

 Insect numbers plunged half parts world due climate change intensive agriculture study foundThe combined pressures global heating farming driving substantial decline insects across globe according UK researchersThey say must acknowledge threats pose insects species lost foreverBut preserving habitat nature could help ensure vital insects thrive 
science
 Harmful bacteria levels protected river 500 times greater downriver sewage treatment works campaigners sayWater firms must end discharges rather trying mitigate problems cause campaigner Matt Staniek saidUnited Utilities runs treatment works said required remove bacteria 
science
 The cost storing bike hangar five times higher car parking permit areas LondonOnly six boroughs offer bike storage cheaper similar rates cost permit 10 boroughs electric vehicles parked free chargeCyclists pay £107 bike hangar usage annually despite taking much less space drivers doSimon Munk London Cycling Campaign said price disparity discouraging cycling 

 For decades Bollywood makes Hindi films seen synonymous Indian films other-language industries termed regional sands shifting little Last week KGF 2 Kannada film also dubbed languages released 4 500 screens across India number usually reserved Bollywood superstars Salman KhanAfter spectacular opening movie sequel flamboyant 2018 hit gangster fights take gold mine looks set become one India biggest box-office hitsKGF 2 came heels Telugu-language blockbuster RRR still drawing crowds northern Indian theatresA couple months ago another Telugu film Pushpa performed well Hindi audience streaming online since dialogues songs sparked hundreds Instagram reelsOver decades many filmmakers attempted make pan-Indian films appeal audiences across country major challenge considering diversity languages cultures India Some like Tamil directors Mani Ratnam Shankar succeeded films addressed larger national anxieties dramatic love stories grand productions featuring big stars efforts intermittent Bollyw

 There much solemnity seriousness Queen life reaches 96th birthday often overlooked sense humour Laughing things important survival technique says royal historian Robert LaceyThe Queen well-developed sense absurd vital private outlet highly choreographed public life often remain straight-faced says She obviously takes job seriously time stop sense ridiculous Mr Lacey saysHistorian author Sir Anthony Seldon says A defining characteristic take seriously It significantly contributed success reign He says Queen none self-obsession calls disease age There lack pomposity certain irreverence says She able laugh misfortune keep going In private Queen said good mimic according Mr Lacey historical consultant Netflix series The CrownShe said particularly good impression former Russian leader Boris Yeltsin says Karen Dol author Wicked Wit Queen Elizabeth IIAnd imitates politicians clerics TV charactersMr Lacey describes Queen humour often self-deprecating mildly making fun He gives example politic

 BBC News NI outlines latest data coronavirus Covid-19 vaccinations across Northern Ireland Republic IrelandNine Covid-19-related deaths reported Northern Ireland ThursdayThe total number deaths linked Covid-19 Northern Ireland since start pandemic 3 402 Deaths measured recording died within 28 days receiving positive result test coronavirus Another 752 cases coronavirus reported Thursday 588 WednesdayThe data includes cases confirmed samples taken previous days necessarily latest reporting periodSince 5 January people Northern Ireland positive lateral flow test LFT longer need polymerase chain reaction PCR test confirm resultHowever Department Health said extremely important people report LFT test results positiveOn Thursday 387 people Covid-19 hospital Northern Ireland five people ICUThat figure Wednesday 380 people Covid-19 hospital Northern Ireland five people ICULast updated 21 April 14:20 BSTSource Department Health Northern IrelandA total 3 778 302 vaccines Covid-19 administered

 A custody nurse Lincoln police station suspended deplorable list misconductPaul Bright admitted failing refer patient head injury hospital properly assess diabetic patient suffered seizureHe also failed properly record medication several occasions lied checks doneThe Nursing Midwifery Council NMC found fitness practise impaired reason misconduct 
health
 Boris Johnson says MPs full facts Partygate ordering inquiry whether misled lockdown partiesMPs set vote Thursday Labour plan Commons committee investigate past comments Whitehall gatheringsBut ministers say vote wait probes Met Police civil servant Sue Gray finishedMr Johnson miss debate India official visitSpeaking trip said keen every possible form scrutiny wanted focus amazing opportunities developing UK IndiaEarlier Mr Johnson told reporters would fight next general election would say circumstances would resignAlong wife Carrie Chancellor Rishi Sunak PM fined breaking Covid laws June 2020 birthday party No 10He previously told MP